In [1]:
import pandas as pd
from tqdm import tqdm
import gensim
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# os.chdir(r"D://Proposal_Defense//Simulations")
# from Utils.Script_utils import get_data_splits, first_LSTM_training, get_data_splits_old_algo
from joblib import Parallel, delayed
import sys
from scipy.stats import norm

In [2]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 42

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# # 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")
# tf.config.set_visible_devices([], 'GPU')

from tqdm.notebook import tqdm
tf.random.set_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
# from keras import backend as K
# session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
# K.set_session(sess)

No GPU found


In [3]:
import matplotlib.pyplot as plt

import pickle

In [4]:
# def get_parent_dir(directory):
#     import os
#     return os.path.dirname(directory)

# current_dirs_parent = get_parent_dir(os.getcwd())

In [5]:
# os.chdir(current_dirs_parent)

In [6]:
# current_dirs_parent

In [7]:
model_cbow = gensim.models.word2vec.Word2Vec.load(r"..//word2vec_sg")

In [8]:
lr = 1e-3
# from sklearn.model_selection import train_test_split


In [9]:
reps = 50

In [10]:
# with open('true_data_doc2vec_lstm_var_0.0001.pkl', 'wb') as f:
#     pickle.dump(catch, f)


with open('..//Data//train_valid_test_splits_50.pkl', 'rb') as f:
    catch = pickle.load(f)  

In [11]:
rate = 0.5

In [12]:
def attention_lstm_model(rate, training): 
    padding_vector = np.zeros((1, model_cbow.wv.vectors.shape[1]))
    unknown_vector = np.zeros((1, model_cbow.wv.vectors.shape[1]))
    weight_vectors = np.vstack((padding_vector, unknown_vector))
    weight_vectors = np.vstack((weight_vectors, model_cbow.wv.vectors))
    embedding_layer = tf.keras.layers.Embedding(len(weight_vectors),
                            weight_vectors.shape[1],
                            weights=[weight_vectors],
                            mask_zero = False,
                            trainable=False)

    
    vectorize_layer = tf.keras.layers.TextVectorization(
                     output_mode='int',
                     vocabulary=model_cbow.wv.index_to_key, 
                     standardize = None)
    
    input_layer = tf.keras.layers.Input(shape = (1,), dtype = tf.string)

    vectorize = vectorize_layer(input_layer)

    vectorize.shape

    emb_output = embedding_layer(vectorize)

    emb_output.shape
    
    # spatial_drop = tf.keras.layers.SpatialDropout1D(0.5)
    
    # spatial_out = spatial_drop(emb_output, training = training)

    lstm_layer = tf.keras.layers.LSTM(25, return_sequences = True, dropout = rate)

    lstm_output = lstm_layer(emb_output, training = training)

#     x_a = tf.keras.layers.Dense(lstm_output.get_shape()[-1]//2, kernel_initializer = 'glorot_uniform', activation="tanh", name="tanh_mlp")(lstm_output) 
    
    # x_a = tf.keras.layers.SpatialDropout1D(0.75)(lstm_output, training = training)
    
    x_a = tf.keras.layers.Dense(1, kernel_initializer = 'glorot_uniform', activation='linear', name="word-level_context", kernel_regularizer=tf.keras.regularizers.L2())(lstm_output)

    x_a = tf.keras.layers.Flatten()(x_a)

    att_out = tf.keras.layers.Activation('softmax')(x_a) 

    x_a2 = tf.keras.layers.RepeatVector(lstm_output.get_shape()[-1])(att_out)

    x_a2 = tf.keras.layers.Permute([2,1])(x_a2)

    out = tf.keras.layers.Multiply()([lstm_output,x_a2])
    
    out = tf.keras.layers.Lambda(lambda x : tf.math.reduce_sum(x, axis = 1), name='expectation_over_words')(out)
    
    # dropout_layer = tf.keras.layers.Dropout(0.65)(out, training = training)

    pred_head = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.L2())

    pred_output = pred_head(out)

    model = tf.keras.models.Model(input_layer, pred_output)
    
    model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                 optimizer = tf.keras.optimizers.Adam(learning_rate = lr), 
                 metrics=tf.keras.metrics.BinaryAccuracy())
    
    return model

In [13]:
# chk = attention_lstm_model(rate, False)

In [14]:
# chk.layers

In [15]:
rate

0.5

In [16]:
def first_LSTM_training(idx):

    first_lstm = attention_lstm_model(rate, False)
    # model_word2vec = simple_lstm(False, model_cbow)
    # init_weights = first_lstm.get_weights()
    
    first_lstm.fit(catch[idx][0], catch[idx][3], epochs = 2000, verbose = 0, 
                  callbacks = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 30,
                                                              restore_best_weights=True), 
                      validation_data = (catch[idx][1], catch[idx][4]))
    
    X_train_logits = first_lstm.predict(catch[idx][0], verbose = 0)
    X_valid_logits = first_lstm.predict(catch[idx][1], verbose = 0)
    X_test_logits = first_lstm.predict(catch[idx][2], verbose = 0)
    
    lstm_extract = tf.keras.models.Model(first_lstm.input, first_lstm.layers[-2].output)
    
    train_lstm_extract = lstm_extract(catch[idx][0])
    valid_lstm_extract = lstm_extract(catch[idx][1])
    test_lstm_extract = lstm_extract(catch[idx][2])
    
    first_lstm.save("Real_World_LSTM_Models_Low_Dropout" + "//" + "Model_" + str(idx))
    
    return X_train_logits, X_valid_logits , X_test_logits,  train_lstm_extract, valid_lstm_extract, test_lstm_extract

In [17]:
first_lstm = Parallel(n_jobs=15, verbose = 10, backend = "loky")(delayed(first_LSTM_training)( i) for i in range(reps))

[Parallel(n_jobs=15)]: Using backend LokyBackend with 15 concurrent workers.
[Parallel(n_jobs=15)]: Done   2 tasks      | elapsed:   15.2s
[Parallel(n_jobs=15)]: Done  11 tasks      | elapsed:   17.2s
[Parallel(n_jobs=15)]: Done  20 tasks      | elapsed:   29.4s
[Parallel(n_jobs=15)]: Done  27 out of  50 | elapsed:   33.8s remaining:   28.8s
[Parallel(n_jobs=15)]: Done  33 out of  50 | elapsed:   42.7s remaining:   22.0s
[Parallel(n_jobs=15)]: Done  39 out of  50 | elapsed:   48.6s remaining:   13.7s
[Parallel(n_jobs=15)]: Done  45 out of  50 | elapsed:   53.5s remaining:    5.9s
[Parallel(n_jobs=15)]: Done  50 out of  50 | elapsed:   56.0s finished


In [18]:
with open('low_dropout_first_lstm.pkl', 'wb') as f:
    pickle.dump(first_lstm, f)

In [19]:
catch_train_logits = []
catch_train_probs = []
catch_train_labels = []
catch_train_word2vec = []
catch_train_seqs = []
# catch_train_word2vec = []
for i in range(0, reps):
    # train_true_probs = tf.math.sigmoid(first_lstm[i][0])
#     idxes = ((train_true_probs >= 0.05) & (train_true_probs <= 0.95)).numpy().nonzero()[0]
#     catch_train_idxes.append(idxes)
    train_logits = first_lstm[i][0]
    catch_train_logits.append(train_logits)
    # train_probs = train_true_probs.numpy()
    # catch_train_probs.append(train_probs)
    # train_labels = catch[i][3]
    # catch_train_labels.append(train_labels)
    # # catch_train_word2vec.append(catch[i][6])
    # catch_train_seqs.append(catch[i][0])

In [20]:
catch_valid_logits = []
catch_valid_probs = []
catch_valid_labels = []
catch_valid_word2vec = []
catch_valid_seqs = []
# catch_train_word2vec = 
for i in range(0, reps):
    # valid_true_probs = tf.math.sigmoid(first_lstm[i][1])
    valid_logits = first_lstm[i][1]
    catch_valid_logits.append(valid_logits)
    # valid_probs = valid_true_probs.numpy()
    # catch_valid_probs.append(valid_probs)
    # valid_labels = catch[i][4]
    # catch_valid_labels.append(valid_labels)
    # # catch_valid_word2vec.append(catch[i][4])
    # catch_valid_seqs.append(catch[i][1])

In [21]:
catch_test_logits = []
catch_test_probs = []
catch_test_labels = []
# catch_test_word2vec = []
catch_test_seqs = []
for i in range(0, reps):
    # test_true_probs = tf.math.sigmoid(first_lstm[i][2])
#     idxes = ((test_true_probs >= 0.05) & (test_true_probs <= 0.95)).numpy().nonzero()[0]
#     catch_test_idxes.append(idxes)
    test_logits = first_lstm[i][2]
    catch_test_logits.append(test_logits)
    # test_probs = test_true_probs.numpy()
    # catch_test_probs.append(test_probs)
    # test_labels = catch[i][5]
    # catch_test_labels.append(test_labels)
    # # catch_test_word2vec.append(catch[i][5])
    # catch_test_seqs.append(catch[i][2])

In [22]:
with open('Low_Data_train_logits.pkl', 'wb') as f:
    pickle.dump(catch_train_logits, f)

In [23]:
with open('Low_Data_valid_logits.pkl', 'wb') as f:
    pickle.dump(catch_valid_logits, f)

In [24]:
with open('Low_Data_test_logits.pkl', 'wb') as f:
    pickle.dump(catch_test_logits, f)

In [25]:
# ens_model = tf.keras.models.load_model("Real_World_LSTM_Models" + "//" + "Model_" + str(0))

In [26]:
# ens_model.summary()

In [27]:
# base_model = attention_lstm_model(True)

In [28]:
# base_model.set_weights(ens_model.get_weights())

In [29]:
# base_model.predict(catch[0][0])